In [ ]:
# Debug a Processed Collection

In [ ]:
%load_ext autoreload
%autoreload 2

from scripts.debug import *
import os

### `parameters`

In [ ]:
root_dir = r"Z:\collections\ebfe-12040101_WestForkSanJacinto"


db_path = os.path.join(root_dir, "ripple.db")
error_report_path = os.path.join(root_dir, "error_report.xlsx")

### Create Excel Error Report

In [ ]:

dfs = []
for process_name in ["extract_submodel", "create_ras_terrain", "create_model_run_normal_depth", "run_incremental_normal_depth", "run_known_wse"]:
    # poll_and_update_job_status(db_path, process_name)
    _, failed_reaches, _ = get_reach_status_by_process(db_path, process_name)
    print("failed_reaches")
    df = get_failed_jobs_df(failed_reaches)
    print("df")
    dfs.append(df)
    write_failed_jobs_df_to_excel(df, process_name, error_report_path)